# Session 5

1. Combiner les données
2. Stratégie split-apply-combine (groupby)
3. Index hiérarchiques
4. Méthodes de reshaping (2)

In [1]:
# imports
import numpy as np
import pandas as pd

**Dataset n° 1 et n° 2**

The GeoNames geographical database covers all countries and contains over eleven million placenames that are available for download free of charge.

https://www.geonames.org/

#### Pays

Voir : https://www.geonames.org/countries/

In [8]:
# dataset 1
var = pd.read_html('https://www.geonames.org/countries/',
                   header=0,
                   keep_default_na=False,  # NA = North America
                   encoding="utf-8"
                  )
[x.shape for x in var]

[(0, 2), (250, 9)]

In [9]:
# pays
df_pays = var[1]
df_pays.head()

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NA
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA


In [10]:
# nombre de pays par continent
df_pays['Continent'].value_counts()

AF    58
EU    53
AS    51
NA    41
OC    28
SA    14
AN     5
Name: Continent, dtype: int64

In [12]:
df_pays.loc[df_pays["Continent"]=="NA", "Country"].unique()

array(['Antigua and Barbuda', 'Anguilla', 'Aruba', 'Barbados',
       'Saint Barthélemy', 'Bermuda', 'Bonaire, Sint Eustatius, and Saba',
       'Bahamas', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Curaçao',
       'Dominica', 'Dominican Republic', 'Grenada', 'Greenland',
       'Guadeloupe', 'Guatemala', 'Honduras', 'Haiti', 'Jamaica',
       'St Kitts and Nevis', 'Cayman Islands', 'Saint Lucia',
       'Saint Martin', 'Martinique', 'Montserrat', 'Mexico', 'Nicaragua',
       'Panama', 'Saint Pierre and Miquelon', 'Puerto Rico',
       'El Salvador', 'Sint Maarten', 'Turks and Caicos Islands',
       'Trinidad and Tobago', 'United States',
       'St Vincent and Grenadines', 'British Virgin Islands',
       'U.S. Virgin Islands'], dtype=object)

In [13]:
df_pays.loc[df_pays["Continent"]=="EU", "Country"].unique()

array(['Andorra', 'Albania', 'Austria', 'Åland', 'Bosnia and Herzegovina',
       'Belgium', 'Bulgaria', 'Belarus', 'Switzerland', 'Cyprus',
       'Czechia', 'Germany', 'Denmark', 'Estonia', 'Spain', 'Finland',
       'Faroe Islands', 'France', 'United Kingdom', 'Guernsey',
       'Gibraltar', 'Greece', 'Croatia', 'Hungary', 'Ireland',
       'Isle of Man', 'Iceland', 'Italy', 'Jersey', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Latvia', 'Monaco', 'Moldova',
       'Montenegro', 'North Macedonia', 'Malta', 'Netherlands', 'Norway',
       'Poland', 'Portugal', 'Romania', 'Serbia', 'Russia', 'Sweden',
       'Slovenia', 'Svalbard and Jan Mayen', 'Slovakia', 'San Marino',
       'Ukraine', 'Vatican City', 'Kosovo'], dtype=object)

#### Villes

Voir : http://download.geonames.org/export/dump/

In [14]:
# villes
df = pd.read_csv('cities500.zip',
                 sep='\t',
                 header=None,
                 keep_default_na=False,  # NA = North America
                 na_values=['', -9999],
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 
                        'longitude', 'feature class', 'feature code', 'country code', 
                        'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                dtype={'admin1 code': str,
                       'admin2 code': str,
                       'admin3 code': str,
                       'admin4 code': str})
df.shape

(199301, 19)

In [15]:
# villes
df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,02,NaN,NaN,NaN,602,NaN,1832.0,Europe/Andorra,2017-11-06
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052,NaN,1721.0,Europe/Andorra,2012-11-03
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022,NaN,921.0,Europe/Andorra,2013-11-23
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363,2050.0,2106.0,Europe/Andorra,2008-06-09
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,05,NaN,NaN,NaN,3066,NaN,1296.0,Europe/Andorra,2018-10-26


In [16]:
# nombre de villes par pays
df['country code'].value_counts().head(16)

US    21135
MX    16929
CN    15611
FR    15269
IT    11739
DE    11514
ID     9224
ES     7213
RO     7148
GB     5710
RU     5023
AU     4888
PH     4471
PL     3677
IN     3551
AT     3024
Name: country code, dtype: int64

**Exercice n° 1**

- Quelle est la ville la plus peuplée (population) ?
- Quelle est la ville la plus haute (elevation ou dem) ?
- Quelle est la ville la plus basse (elevation ou dem) ?
- La colonne "alternatenames" contient pour chaque ville les différents noms de chacune des villes, séparés par des virgules. Quelle ville possède le plus de noms différents ? Donnez la liste des noms.

In [24]:
%timeit df.sort_values(by="population", ascending=False)["name"].head(1).values[0]

126 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit df.nlargest(1, "population")["name"].values[0]

46.9 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit df.loc[df["population"].idxmax(), "name"]

192 µs ± 7.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
df.loc[df["dem"].idxmax(), "name"]

'Zongga'

In [26]:
df.loc[df["elevation"].idxmax(), "name"]

'Thang'

In [27]:
df.loc[df["elevation"].idxmin(), "name"]

'Desert Shores'

In [39]:
%timeit df["alternatenames"].dropna().str.split(",").apply(lambda x: len(x)).idxmax()

308 ms ± 7.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit df["alternatenames"].dropna().str.count(",").idxmax()

207 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
print(*df.loc[df["alternatenames"].dropna().str.split(",").apply(lambda x: len(x)).idxmax(), "alternatenames"].split(","))

Adonai-jireh Aelia Capitolina Al Quds Al-Kuds Al-Qudis Al-Quds Al-Qudıs Ariel Baitul Maqdis Baitulmuqaddis Bajtul Makudis Bayt al-Maqdis Bayt al-Muqaddas Cherusalem City of David Colonia Aelia Capitolina Djeruzalem El Kuds El Quds esh Sherif Erusagem Erusalim Geruesalem Gerusalem Gerusalemme Gerüsalem Gierusalemme Girusalemmi Herusale Herusalem Herusalẽ Hierosolyma Hierousalem Hierousalēm Hierusalem Hiruharama IJerusalem Ia-lo-sak-leng Ia-lo-sat-leng Iarusaileim Iarúsailéim Iebous Iebus Ierosolyma Ierousalem Ierousalimu Ierusalem Ierusalema Ierusalim Ir David Ir Ha-Qdoosha Ir Ha-Qodesh Iyerusalim Ià-lô-sák-lēng Iâ-lō͘-sat-léng Iýerusalim JRS Jarusalen Jeriwsalem Jerozale Jerozalė Jerozolema Jerozolima Jerozolëma Jerusalem Jerusalemi Jerusalemo Jerusalemu Jerusalen Jerusalim Jerusalèm Jerusalém Jerusalén Jeruusalemm Jeruzale Jeruzalem Jeruzaleme Jeruzalim Jeruzalém Jeruzalė Jeruzsalem Jeruzsálem Jerúsalem Jerúsálẹ́mù Jorsala Jorsalaborg Jorsalir Jérusalem Jórsalaborg Jórsalir Kouds Kudu

### 1. Combiner les données

Union ensembliste :
- fonction `concat()` : concaténation de Series ou DataFrames à partir d'une liste
- <strike>à noter la méthode `append()` est à présent *deprecated*</strike>

Jointure de bases de données :
- fonction `merge()` : jointure de 2 DataFrames ('on' ou bien 'left_on' + 'right_on', 'how', 'suffixes' pour les colonnes dupliquées)
- méthode `join()` : jointure d'un DataFrame à un autre ('on', 'how', 'rsuffix', 'lsuffix' pour les colonnes dupliquées)


`pd.merge(df1, df2) <=> df1.join(df2)`



Mot-clé 'how' :
- 'inner' (SQL INNER JOIN) : intersection des valeurs des 2 colonnes de jointure
- 'left' (SQL LEFT OUTER JOIN) : valeurs de la colonne de jointure de gauche
- 'right' (SQL RIGHT OUTER JOIN) : valeurs de la colonne de jointure de droite
- 'outer' (SQL OUTER JOIN) : union des valeurs des 2 colonnes de jointure
- 'cross' (SQL CROSS JOIN) : produit cartésien des 2 colonnes de jointure

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [46]:
df

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,02,NaN,NaN,NaN,602,NaN,1832.0,Europe/Andorra,2017-11-06
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052,NaN,1721.0,Europe/Andorra,2012-11-03
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022,NaN,921.0,Europe/Andorra,2013-11-23
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363,2050.0,2106.0,Europe/Andorra,2008-06-09
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,05,NaN,NaN,NaN,3066,NaN,1296.0,Europe/Andorra,2018-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199296,895269,Beitbridge,Beitbridge,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",-22.21667,30.00000,P,PPL,ZW,NaN,07,NaN,NaN,NaN,26459,NaN,461.0,Africa/Harare,2013-03-12
199297,895308,Beatrice,Beatrice,Beatrice,-18.25283,30.84730,P,PPL,ZW,NaN,04,NaN,NaN,NaN,1647,NaN,1307.0,Africa/Harare,2018-05-09
199298,895417,Banket,Banket,"Banket,Banket Junction",-17.38333,30.40000,P,PPL,ZW,NaN,05,NaN,NaN,NaN,9641,NaN,1277.0,Africa/Harare,2013-03-12
199299,1085510,Epworth,Epworth,Epworth,-17.89000,31.14750,P,PPLX,ZW,NaN,10,NaN,NaN,NaN,123250,NaN,1508.0,Africa/Harare,2012-01-19


In [48]:
df_pays

,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NA
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA
...,...,...,...,...,...,...,...,...,...
245,YE,YEM,887,YM,Yemen,Sanaa,527970.0,28498687,AS
246,YT,MYT,175,MF,Mayotte,Mamoudzou,374.0,279471,AF
247,ZA,ZAF,710,SF,South Africa,Pretoria,1219912.0,57779622,AF
248,ZM,ZMB,894,ZA,Zambia,Lusaka,752614.0,17351822,AF


In [47]:
# merge df et df_pays avec le code iso2
df = pd.merge(df,
              df_pays,
              left_on='country code',
              right_on='ISO-3166alpha2',
              how='left')
df.to_pickle('df_geo.pkl')
df.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,modification date,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,...,2017-11-06,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
1,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,...,2012-11-03,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
2,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,...,2013-11-23,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
3,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,...,2008-06-09,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU
4,3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,...,2018-10-26,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU


In [ ]:
pd.read_pickle('df_geo.pkl')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199301 entries, 0 to 199300
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   geonameid          199301 non-null  int64  
 1   name               199301 non-null  object 
 2   asciiname          199284 non-null  object 
 3   alternatenames     163478 non-null  object 
 4   latitude           199301 non-null  float64
 5   longitude          199301 non-null  float64
 6   feature class      199301 non-null  object 
 7   feature code       199300 non-null  object 
 8   country code       199301 non-null  object 
 9   cc2                6249 non-null    object 
 10  admin1 code        199268 non-null  object 
 11  admin2 code        168285 non-null  object 
 12  admin3 code        90365 non-null   object 
 13  admin4 code        32608 non-null   object 
 14  population         199301 non-null  int64  
 15  elevation          33491 non-null   float64
 16  de

In [50]:
# vérification de l'identité des 2 colonnes (INUTILE !)
(df['country code'] == df['ISO-3166alpha2']).all()

True

In [51]:
# nombre de villes par continent
df['Continent'].value_counts()

EU    93964
AS    44379
NA    42571
SA     7389
OC     5948
AF     5047
AN        3
Name: Continent, dtype: int64

### 2. Stratégie split-apply-combine

La stratégie split-apply-combine consiste à :
- éclater les données en sous-groupes sur la base d'un critère (par ex., les valeurs d'une colonne)
- appliquer une fonction à chacun des sous-groupes indépendamment
- combiner les résultats en une structure de données

`df.groupby()` retourne un objet de type DataFrameGroupBy qui peut être vu comme un dictionnaire dont les clés sont les différentes valeurs de la colonne utilisée pour éclater les données, et dont les valeurs sont des sous-DataFrames ou des sous-Series correspondant aux données éclatées.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [52]:
# groupby Continent
cont_group = df.groupby('Continent')
type(cont_group)

pandas.core.groupby.generic.DataFrameGroupBy

In [53]:
# ngroups
cont_group.ngroups

7

In [54]:
# sous-groupe
cont_group.get_group('AN')

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,modification date,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
1091,6696480,McMurdo Station,McMurdo Station,"Base McMurdo,Base antarctique McMurdo,Estacao ...",-77.84632,166.66824,P,PPL,AQ,NaN,...,2022-02-04,AQ,ATA,10,AY,Antarctica,,14000000.0,0,AN
83834,3426466,Grytviken,Grytviken,"Gritviken,Griutvikenas,Grjutviken,Grutviken,Gr...",-54.28111,-36.50920,P,PPLC,GS,GB,...,2018-07-15,GS,SGS,239,SX,South Georgia and South Sandwich Islands,Grytviken,3903.0,30,AN
168994,1546102,Port-aux-Français,Port-aux-Francais,"Port o Franse,Port o Fransė,Port-aux-Francais,...",-49.34916,70.21937,P,PPLC,TF,FR,...,2018-08-17,TF,ATF,260,FS,French Southern Territories,Port-aux-Francais,7829.0,140,AN


### Application de méthodes standard

- `size()` : group sizes
- `count()` : count of group
- `mean()` : mean of groups
- `sum()` : sum of group values
- `std()` : standard deviation of groups
- `var()` : variance of groups
- `first()` : first of group values
- `last()` : last of group values
- `nth()` : nth value
- `min()` : min of group values
- `max()` : max of group values

In [55]:
# exemple avec size()
df.groupby('Continent').size()

Continent
AF     5047
AN        3
AS    44379
EU    93964
NA    42571
OC     5948
SA     7389
dtype: int64

In [56]:
# exemple avec size()
df.groupby('Country').size()

Country
Afghanistan       314
Albania           357
Algeria           266
American Samoa     23
Andorra            12
                 ... 
Western Sahara      4
Yemen             314
Zambia             67
Zimbabwe           66
Åland              16
Length: 246, dtype: int64

In [58]:
# exemple avec sum()
df.groupby('Continent')['population'].sum()

Continent
AF     337953065
AN          1305
AS    1721484518
EU     708474746
NA     469001480
OC      49376730
SA     294276793
Name: population, dtype: int64

In [59]:
# exemple avec sum()
df.groupby('Continent').sum()

,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
Continent,,,,,,,,,
AF,9965250563,2.478842e+04,8.331383e+04,337953065,24869.0,2763604.0,2529227,3.921496e+09,312535380743
AN,11669048,-1.814766e+02,2.003784e+02,1305,10.0,15.0,509,1.401173e+07,170
AS,153622187262,9.280883e+05,4.470400e+06,1721484518,153093.0,17075833.0,15752356,1.869690e+11,30513727394690
EU,265779678136,4.492731e+06,1.190148e+06,708474746,3471717.0,23908807.0,42015287,1.139010e+11,4745169464586
NA,219493163859,1.266100e+06,-3.959442e+06,469001480,6171472.0,28936694.0,27414708,2.505692e+11,9123720078452
OC,32275137873,-1.839641e+05,8.298684e+05,49376730,4093.0,677925.0,751893,3.775796e+10,125582947974
SA,29037057765,-9.254385e+04,-4.712113e+05,294276793,49636.0,7270308.0,2224374,2.478581e+10,612773990122


In [61]:
df.select_dtypes(include=np.number)

,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
0,3038999,42.57688,1.66769,602,NaN,1832.0,20,468.0,77006
1,3039154,42.57952,1.65362,1052,NaN,1721.0,20,468.0,77006
2,3039163,42.46372,1.49129,8022,NaN,921.0,20,468.0,77006
3,3039604,42.54277,1.73361,2363,2050.0,2106.0,20,468.0,77006
4,3039678,42.55623,1.53319,3066,NaN,1296.0,20,468.0,77006
...,...,...,...,...,...,...,...,...,...
199296,895269,-22.21667,30.00000,26459,NaN,461.0,716,390580.0,14439018
199297,895308,-18.25283,30.84730,1647,NaN,1307.0,716,390580.0,14439018
199298,895417,-17.38333,30.40000,9641,NaN,1277.0,716,390580.0,14439018
199299,1085510,-17.89000,31.14750,123250,NaN,1508.0,716,390580.0,14439018


### Méthode aggregate() ou agg()

Applique une fonction, une liste de fonctions ou un dictionnaire de fonctions à un groupby.

#### Avec une fonction simple

In [62]:
# exemple
df.groupby('Continent').agg('mean')  # idem que df.groupby('Continent').mean()

,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
Continent,,,,,,,,,
AF,1.974490e+06,4.911516,16.507594,66961.177927,621.725000,548.769658,501.134734,7.769954e+05,6.192498e+07
AN,3.889683e+06,-60.492197,66.792803,435.000000,10.000000,7.500000,169.666667,4.670577e+06,5.666667e+01
AS,3.461596e+06,20.912781,100.732335,38790.520697,484.471519,385.189439,354.950675,4.213005e+06,6.875713e+08
EU,2.828527e+06,47.813326,12.666001,7539.852986,297.007186,254.712111,447.142384,1.212177e+06,5.049987e+07
NA,5.155932e+06,29.740911,-93.007954,11016.924197,290.176415,680.335128,643.976134,5.885914e+06,2.143177e+08
OC,5.426217e+06,-30.928735,139.520581,8301.400471,50.530864,114.417722,126.411063,6.348009e+06,2.111347e+07
SA,3.929768e+06,-12.524543,-63.772000,39826.335499,517.041667,984.203059,301.038571,3.354420e+06,8.293057e+07


#### Avec une liste de fonctions

In [63]:
# exemple
df.groupby('Continent').agg(['mean', 'std'])

geonameid                 latitude              longitude  \
                   mean           std       mean        std        mean   
Continent                                                                 
AF         1.974490e+06  1.866855e+06   4.911516  18.382359   16.507594   
AN         3.889683e+06  2.606247e+06 -60.492197  15.230077   66.792803   
AS         3.461596e+06  3.191502e+06  20.912781  17.080973  100.732335   
EU         2.828527e+06  1.856117e+06  47.813326   5.108585   12.666001   
NA         5.155932e+06  1.751363e+06  29.740911  11.163139  -93.007954   
OC         5.426217e+06  3.323573e+06 -30.928735   9.772017  139.520581   
SA         3.929768e+06  1.254694e+06 -12.524543  13.862533  -63.772000   

                         population                  elevation              \
                  std          mean            std        mean         std   
Continent                                                                    
AF          18.551475  66961.177927  370688.762528  621.725000  706.274516   
AN         101.632052    435.000000     713.063111   10.000000         NaN   
AS          27.711691  38790.520697  351928.129036  484.471519  694.914367   
EU          15.992626   7539.852986   64145.293169  297.007186  284.110412   
NA          13.509162  11016.924197   95787.733946  290.176415  387.792365   
OC          52.213807   8301.400471   92842.484803   50.530864  115.372558   
SA          13.345195  39826.335499  279102.063845  517.041667  750.271767   

                  dem              ISO-3166numeric               Area in km²  \
                 mean          std            mean         std          mean   
Continent                                                                      
AF         548.769658   558.390106      501.134734  250.189004  7.769954e+05   
AN           7.500000     4.949747      169.666667  138.673477  4.670577e+06   
AS         385.189439   615.797691      354.950675  209.457480  4.213005e+06   
EU         254.712111   258.142187      447.142384  234.001545  1.212177e+06   
NA         680.335128   806.194904      643.976134  209.821352  5.885914e+06   
OC         114.417722   191.136654      126.411063  205.128055  6.348009e+06   
SA         984.203059  1201.731707      301.038571  280.094269  3.354420e+06   

                           Population                
                    std          mean           std  
Continent                                            
AF         6.099327e+05  6.192498e+07  6.755496e+07  
AN         8.079517e+06  5.666667e+01  7.371115e+01  
AS         4.055727e+06  6.875713e+08  6.265074e+08  
EU         3.779608e+06  5.049987e+07  3.462704e+07  
NA         4.010102e+06  2.143177e+08  1.165782e+08  
OC         2.875882e+06  2.111347e+07  8.397945e+06  
SA         3.241446e+06  8.293057e+07  7.885812e+07

In [64]:
# exemple
df.set_index('Continent').select_dtypes(include=np.number).reset_index().groupby('Continent').agg(['mean', 'std'])

geonameid                 latitude              longitude  \
                   mean           std       mean        std        mean   
Continent                                                                 
AF         1.974490e+06  1.866855e+06   4.911516  18.382359   16.507594   
AN         3.889683e+06  2.606247e+06 -60.492197  15.230077   66.792803   
AS         3.461596e+06  3.191502e+06  20.912781  17.080973  100.732335   
EU         2.828527e+06  1.856117e+06  47.813326   5.108585   12.666001   
NA         5.155932e+06  1.751363e+06  29.740911  11.163139  -93.007954   
OC         5.426217e+06  3.323573e+06 -30.928735   9.772017  139.520581   
SA         3.929768e+06  1.254694e+06 -12.524543  13.862533  -63.772000   

                         population                  elevation              \
                  std          mean            std        mean         std   
Continent                                                                    
AF          18.551475  66961.177927  370688.762528  621.725000  706.274516   
AN         101.632052    435.000000     713.063111   10.000000         NaN   
AS          27.711691  38790.520697  351928.129036  484.471519  694.914367   
EU          15.992626   7539.852986   64145.293169  297.007186  284.110412   
NA          13.509162  11016.924197   95787.733946  290.176415  387.792365   
OC          52.213807   8301.400471   92842.484803   50.530864  115.372558   
SA          13.345195  39826.335499  279102.063845  517.041667  750.271767   

                  dem              ISO-3166numeric               Area in km²  \
                 mean          std            mean         std          mean   
Continent                                                                      
AF         548.769658   558.390106      501.134734  250.189004  7.769954e+05   
AN           7.500000     4.949747      169.666667  138.673477  4.670577e+06   
AS         385.189439   615.797691      354.950675  209.457480  4.213005e+06   
EU         254.712111   258.142187      447.142384  234.001545  1.212177e+06   
NA         680.335128   806.194904      643.976134  209.821352  5.885914e+06   
OC         114.417722   191.136654      126.411063  205.128055  6.348009e+06   
SA         984.203059  1201.731707      301.038571  280.094269  3.354420e+06   

                           Population                
                    std          mean           std  
Continent                                            
AF         6.099327e+05  6.192498e+07  6.755496e+07  
AN         8.079517e+06  5.666667e+01  7.371115e+01  
AS         4.055727e+06  6.875713e+08  6.265074e+08  
EU         3.779608e+06  5.049987e+07  3.462704e+07  
NA         4.010102e+06  2.143177e+08  1.165782e+08  
OC         2.875882e+06  2.111347e+07  8.397945e+06  
SA         3.241446e+06  8.293057e+07  7.885812e+07

#### Avec un dictionnaire de fonctions

In [65]:
# exemple avec un dictionnaire
df.groupby('Continent').agg({'population': 'sum',
                             'elevation': 'mean',
                             'Country': ['min', 'max']})

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          337953065  621.725000         Algeria   
AN               1305   10.000000      Antarctica   
AS         1721484518  484.471519     Afghanistan   
EU          708474746  297.007186         Albania   
NA          469001480  290.176415        Anguilla   
OC           49376730   50.530864  American Samoa   
SA          294276793  517.041667       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                            Åland  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

### Méthode apply()

Applique une lambda ou une fonction définie à un groupby.

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199301 entries, 0 to 199300
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   geonameid          199301 non-null  int64  
 1   name               199301 non-null  object 
 2   asciiname          199284 non-null  object 
 3   alternatenames     163478 non-null  object 
 4   latitude           199301 non-null  float64
 5   longitude          199301 non-null  float64
 6   feature class      199301 non-null  object 
 7   feature code       199300 non-null  object 
 8   country code       199301 non-null  object 
 9   cc2                6249 non-null    object 
 10  admin1 code        199268 non-null  object 
 11  admin2 code        168285 non-null  object 
 12  admin3 code        90365 non-null   object 
 13  admin4 code        32608 non-null   object 
 14  population         199301 non-null  int64  
 15  elevation          33491 non-null   float64
 16  de

In [67]:
# retourne la ville la plus peuplée d'un DataFrame
def top1city(group):
    return group.loc[group['population'].idxmax(), 'name']

In [68]:
# avec df
top1city(df)

'Shanghai'

In [69]:
# avec le groupby Continent
df.groupby('Continent').apply(top1city)

Continent
AF              Lagos
AN    McMurdo Station
AS           Shanghai
EU             Moscow
NA        Mexico City
OC             Sydney
SA       Buenos Aires
dtype: object

In [70]:
# avec le groupby Country
df.groupby('Country').apply(top1city)

Country
Afghanistan                  Kabul
Albania                     Tirana
Algeria                    Algiers
American Samoa           Pago Pago
Andorra           Andorra la Vella
                        ...       
Western Sahara            Laayoune
Yemen                        Sanaa
Zambia                      Lusaka
Zimbabwe                    Harare
Åland                    Mariehamn
Length: 246, dtype: object

In [77]:
# retourne les 3 villes les plus peuplées d'un DataFrame
def top3city(group):
    return group.nlargest(3, 'population')[['name', 'population']]

In [78]:
# avec df
top3city(df)

,name,population
23334,Shanghai,22315474
27374,Beijing,18960744
23188,Shenzhen,17494398


In [80]:
# par continent
df.groupby('Continent').apply(top3city).droplevel(1)  # supprime le multi-index

,name,population
Continent,,
AF,Lagos,15388000
AF,Cairo,9606916
AF,Kinshasa,7785965
AN,McMurdo Station,1258
AN,Port-aux-Français,45
AN,Grytviken,2
AS,Shanghai,22315474
AS,Beijing,18960744
AS,Shenzhen,17494398


**Exercice n° 2**

Créez une fonction qui calcule la moyenne du nombre de noms alternatifs de chaque ville.

Testez-la sur tout le dataset.

Appliquez cette fonction sur un groupby 'feature code'. Quel code obtient la valeur la plus grande ?

Voir les features codes des pays : https://www.geonames.org/export/codes.html#P

In [84]:
def mean_alternatenames(group):
    return group["alternatenames"].fillna("").str.count(",").mean() +1

mean_alternatenames(df)

4.421136873372436

In [ ]:
# pandas where mask

In [85]:
df.groupby("feature code").apply(mean_alternatenames)

feature code
PPL       3.697770
PPLA     17.678758
PPLA2     6.805262
PPLA3     3.944806
PPLA4     4.696315
PPLA5     1.428571
PPLC     49.954357
PPLCH     2.000000
PPLF      1.653846
PPLG     11.000000
PPLH      3.333333
PPLL      2.026178
PPLQ      2.440000
PPLR      2.666667
PPLS      2.263158
PPLW      6.166667
PPLX      1.613100
STLMT     6.195652
dtype: float64

In [87]:
df.loc[df["feature code"]=="PPLG"]

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,modification date,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
12540,2800461,Chasse Royale,Chasse Royale,NaN,50.42842,3.95001,P,PPLG,BE,NaN,...,2017-03-25,BE,BEL,56,BE,Belgium,Brussels,30510.0,11422068,EU
13342,2394819,Cotonou,Cotonou,"Appi,COO,Catonou,Cotonou,Cotonu,Cotonú,Katanu,...",6.36536,2.41833,P,PPLG,BJ,NaN,...,2019-09-05,BJ,BEN,204,BN,Benin,Porto-Novo,112620.0,11485048,AF
13466,3911925,La Paz,La Paz,"A Paz,Chukiyawu,Chuqi Yapu,Chuqiyapu,Ciudad La...",-16.50000,-68.15000,P,PPLG,BO,NaN,...,2021-11-27,BO,BOL,68,BL,Bolivia,Sucre,1098580.0,11353142,SA
15555,6693804,Cambebba,Cambebba,NaN,-3.62092,-38.82913,P,PPLG,BR,NaN,...,2012-08-03,BR,BRA,76,BR,Brazil,Brasilia,8511965.0,209469333,SA
35447,12492648,"Guiyang, Guizhou","Guiyang, Guizhou",NaN,26.64590,106.63092,P,PPLG,CN,NaN,...,2022-07-10,CN,CHN,156,CH,China,Beijing,9596960.0,1411778724,AS
51000,7302786,Haselbachtal,Haselbachtal,NaN,51.23570,14.02576,P,PPLG,DE,NaN,...,2015-09-05,DE,DEU,276,GM,Germany,Berlin,357021.0,82927922,EU
51024,8378906,Burg Unter-Falkenstein,Burg Unter-Falkenstein,Neu-Falkenstein,47.70440,12.10466,P,PPLG,DE,NaN,...,2012-08-24,DE,DEU,276,GM,Germany,Berlin,357021.0,82927922,EU
51125,12492033,Altstadt,Altstadt,NaN,49.75493,6.63984,P,PPLG,DE,NaN,...,2022-07-01,DE,DEU,276,GM,Germany,Berlin,357021.0,82927922,EU
84186,4043547,Guam Government House,Guam Government House,Designer Architect Richard Neutra,13.47191,144.74978,P,PPLG,GU,NaN,...,2011-07-01,GU,GUM,316,GQ,Guam,Hagatna,549.0,165768,OC
113336,12032166,bishopric of Perugia,bishopric of Perugia,NaN,43.11258,12.38904,P,PPLG,IT,NaN,...,2019-03-03,IT,ITA,380,IT,Italy,Rome,301230.0,60431283,EU


### 3. Index hiérarchiques

pandas est capable de gérer des index hiérarchiques.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

#### 3.1 Index

In [88]:
# groupby 2 colonnes
# nombre de villes par pays et par continent
s = df.groupby(['Continent', 'Country']).size()
s

Continent  Country     
AF         Algeria          266
           Angola            38
           Benin             47
           Botswana         106
           Burkina Faso      80
                           ... 
SA         Paraguay         163
           Peru            1955
           Suriname          15
           Uruguay          144
           Venezuela        408
Length: 246, dtype: int64

In [89]:
# type de l'index
type(s.index)

pandas.core.indexes.multi.MultiIndex

In [90]:
# niveaux de l'index : Continent, Country
s.index.levels

FrozenList([['AF', 'AN', 'AS', 'EU', 'NA', 'OC', 'SA'], ['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius, and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo Republic', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia', 'DR Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Falkl

In [91]:
# accès au premier niveau
s.loc['AF'] # ou s.loc[('AF',)]

Country
Algeria                     266
Angola                       38
Benin                        47
Botswana                    106
Burkina Faso                 80
Burundi                      21
Cabo Verde                   25
Cameroon                    123
Central African Republic     36
Chad                         50
Comoros                     115
Congo Republic               20
DR Congo                     71
Djibouti                     13
Egypt                       150
Equatorial Guinea            26
Eritrea                      13
Eswatini                     23
Ethiopia                    108
Gabon                        27
Ghana                        79
Guinea                       40
Guinea-Bissau                17
Ivory Coast                 118
Kenya                       117
Lesotho                      12
Liberia                      19
Libya                        55
Madagascar                  124
Malawi                       36
Mali                         67


In [92]:
# accès au second niveau
s.loc[('AF', 'Algeria')]

266

In [93]:
# reset Continent
s.reset_index('Continent') # ou s.reset_index(level=0)

,Continent,0
Country,,
Algeria,AF,266
Angola,AF,38
Benin,AF,47
Botswana,AF,106
Burkina Faso,AF,80
...,...,...
Paraguay,SA,163
Peru,SA,1955
Suriname,SA,15


In [94]:
# reset Country
s.reset_index('Country') # ou s.reset_index(level=1)

,Country,0
Continent,,
AF,Algeria,266
AF,Angola,38
AF,Benin,47
AF,Botswana,106
AF,Burkina Faso,80
...,...,...
SA,Paraguay,163
SA,Peru,1955
SA,Suriname,15


In [95]:
# reset Country
s.reset_index('Country').reset_index() # ou s.reset_index(level=1)

,Continent,Country,0
0,AF,Algeria,266
1,AF,Angola,38
2,AF,Benin,47
3,AF,Botswana,106
4,AF,Burkina Faso,80
...,...,...,...
241,SA,Paraguay,163
242,SA,Peru,1955
243,SA,Suriname,15
244,SA,Uruguay,144


In [97]:
# swaplevel
s.swaplevel()

Country       Continent
Algeria       AF            266
Angola        AF             38
Benin         AF             47
Botswana      AF            106
Burkina Faso  AF             80
                           ... 
Paraguay      SA            163
Peru          SA           1955
Suriname      SA             15
Uruguay       SA            144
Venezuela     SA            408
Length: 246, dtype: int64

In [98]:
# droplevel
s.droplevel(1)

Continent
AF     266
AF      38
AF      47
AF     106
AF      80
      ... 
SA     163
SA    1955
SA      15
SA     144
SA     408
Length: 246, dtype: int64

In [100]:
 df.groupby(['Continent', 'Country'], as_index=False).mean()

,Continent,Country,geonameid,latitude,longitude,population,elevation,dem,ISO-3166numeric,Area in km²,Population
0,AF,Algeria,2.742590e+06,35.450063,3.475167,71585.180451,594.444444,436.548872,12.0,2381740.0,42228429.0
1,AF,Angola,2.861154e+06,-11.004780,16.075051,124744.315789,NaN,962.789474,24.0,1246700.0,30809762.0
2,AF,Benin,2.783211e+06,7.941355,2.275789,64301.851064,NaN,162.510638,204.0,112620.0,11485048.0
3,AF,Botswana,1.017653e+06,-22.938353,25.588591,10259.405660,NaN,1016.707547,72.0,600370.0,2254126.0
4,AF,Burkina Faso,2.582757e+06,12.477639,-1.614048,31038.075000,NaN,307.275000,854.0,274200.0,19751535.0
...,...,...,...,...,...,...,...,...,...,...,...
241,SA,Paraguay,3.543396e+06,-25.455428,-56.583490,24759.294479,434.000000,170.552147,600.0,406750.0,6956071.0
242,SA,Peru,4.469107e+06,-10.550619,-75.677475,11694.506905,1174.400000,2102.348338,604.0,1285220.0,31989256.0
243,SA,Suriname,3.604237e+06,5.528922,-55.267243,21137.200000,NaN,18.000000,740.0,163270.0,575991.0
244,SA,Uruguay,3.616430e+06,-33.504781,-56.241751,18785.888889,0.000000,74.562500,858.0,176220.0,3449299.0


#### 3.2 Columns

In [101]:
# retour sur un dataframe avec des colonnes hiérarchiques
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          337953065  621.725000         Algeria   
AN               1305   10.000000      Antarctica   
AS         1721484518  484.471519     Afghanistan   
EU          708474746  297.007186         Albania   
NA          469001480  290.176415        Anguilla   
OC           49376730   50.530864  American Samoa   
SA          294276793  517.041667       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                            Åland  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

In [102]:
# columns
tab.columns

MultiIndex([('population',  'sum'),
            ( 'elevation', 'mean'),
            (   'Country',  'min'),
            (   'Country',  'max')],
           )

In [103]:
# accès au premier niveau
tab.loc[:, 'Country']  # ou tab.loc[:, ('Country',)]

,min,max
Continent,,
AF,Algeria,Zimbabwe
AN,Antarctica,South Georgia and South Sandwich Islands
AS,Afghanistan,Yemen
EU,Albania,Åland
NA,Anguilla,United States
OC,American Samoa,Wallis and Futuna
SA,Argentina,Venezuela


In [104]:
# accès au second niveau
tab.loc[:, ('Country', 'min')]

Continent
AF           Algeria
AN        Antarctica
AS       Afghanistan
EU           Albania
NA          Anguilla
OC    American Samoa
SA         Argentina
Name: (Country, min), dtype: object

In [105]:
# swaplevel
tab.swaplevel(axis=1)

,sum,mean,min,max
,population,elevation,Country,Country
Continent,,,,
AF,337953065,621.725000,Algeria,Zimbabwe
AN,1305,10.000000,Antarctica,South Georgia and South Sandwich Islands
AS,1721484518,484.471519,Afghanistan,Yemen
EU,708474746,297.007186,Albania,Åland
NA,469001480,290.176415,Anguilla,United States
OC,49376730,50.530864,American Samoa,Wallis and Futuna
SA,294276793,517.041667,Argentina,Venezuela


In [106]:
# droplevel
tab.droplevel(0, axis=1)

,sum,mean,min,max
Continent,,,,
AF,337953065,621.725000,Algeria,Zimbabwe
AN,1305,10.000000,Antarctica,South Georgia and South Sandwich Islands
AS,1721484518,484.471519,Afghanistan,Yemen
EU,708474746,297.007186,Albania,Åland
NA,469001480,290.176415,Anguilla,United States
OC,49376730,50.530864,American Samoa,Wallis and Futuna
SA,294276793,517.041667,Argentina,Venezuela


### 4. Méthodes de reshaping (2)

**pandas** possède plusieurs méthodes de reshaping qui généralisent les méthodes de pivot :
- `stack()` : move the inner-most (or the specified) column level to the inner-most index level
- `unstack()` : move the inner-most (or the specified) index level to the inner-most column level
- `swaplevel()` : swap 2 levels from index or from columns
- `droplevel()` : drop a level of an index or of a column

D'autres méthodes permettent de manipuler les index et peuvent être combinées :
- `set_index()` : move the specified column as the new index
- `reset_index()` : move the specified index as a new column
- `reindex()` : conform to a new index

Enfin, la méthode `melt()` permet de faire passer une table d'un format large vers un format long.

Toutes ces méthodes sont très utiles pour reformatter des data, notamment lorsqu'elles sont fournies dans un format pour les humains et doivent être transformées dans un format pour les machines.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

Voir également l'article Tidy Data.

In [107]:
# exemple
s = df.groupby(['Continent', 'Country']).size()
s

Continent  Country     
AF         Algeria          266
           Angola            38
           Benin             47
           Botswana         106
           Burkina Faso      80
                           ... 
SA         Paraguay         163
           Peru            1955
           Suriname          15
           Uruguay          144
           Venezuela        408
Length: 246, dtype: int64

In [108]:
# exemple
s.unstack()

Country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua and Barbuda,Argentina,...,Vanuatu,Vatican City,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Åland
Continent,,,,,,,,,,,,,,,,,,,,,
AF,NaN,NaN,266.0,NaN,NaN,38.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,67.0,66.0,NaN
AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AS,314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,646.0,NaN,NaN,314.0,NaN,NaN,NaN
EU,NaN,357.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
NA,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OC,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003.0,...,NaN,NaN,408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# exemple
s.unstack('Continent')

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,314.0,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,357.0,NaN,NaN,NaN
Algeria,266.0,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,23.0,NaN
Andorra,NaN,NaN,NaN,12.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,4.0,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,314.0,NaN,NaN,NaN,NaN
Zambia,67.0,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
# exemple
s.unstack('Continent').T

Country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua and Barbuda,Argentina,...,Vanuatu,Vatican City,Venezuela,Vietnam,Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Åland
Continent,,,,,,,,,,,,,,,,,,,,,
AF,NaN,NaN,266.0,NaN,NaN,38.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,67.0,66.0,NaN
AN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AS,314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,646.0,NaN,NaN,314.0,NaN,NaN,NaN
EU,NaN,357.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
NA,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OC,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
SA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003.0,...,NaN,NaN,408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
# exemple
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

population   elevation         Country  \
                  sum        mean             min   
Continent                                           
AF          337953065  621.725000         Algeria   
AN               1305   10.000000      Antarctica   
AS         1721484518  484.471519     Afghanistan   
EU          708474746  297.007186         Albania   
NA          469001480  290.176415        Anguilla   
OC           49376730   50.530864  American Samoa   
SA          294276793  517.041667       Argentina   

                                                     
                                                max  
Continent                                            
AF                                         Zimbabwe  
AN         South Georgia and South Sandwich Islands  
AS                                            Yemen  
EU                                            Åland  
NA                                    United States  
OC                                Wallis and Futuna  
SA                                        Venezuela

In [112]:
# exemple
tab.stack()

Country   elevation  \
Continent                                                              
AF        max                                   Zimbabwe         NaN   
          mean                                       NaN  621.725000   
          min                                    Algeria         NaN   
          sum                                        NaN         NaN   
AN        max   South Georgia and South Sandwich Islands         NaN   
          mean                                       NaN   10.000000   
          min                                 Antarctica         NaN   
          sum                                        NaN         NaN   
AS        max                                      Yemen         NaN   
          mean                                       NaN  484.471519   
          min                                Afghanistan         NaN   
          sum                                        NaN         NaN   
EU        max                                      Åland         NaN   
          mean                                       NaN  297.007186   
          min                                    Albania         NaN   
          sum                                        NaN         NaN   
NA        max                              United States         NaN   
          mean                                       NaN  290.176415   
          min                                   Anguilla         NaN   
          sum                                        NaN         NaN   
OC        max                          Wallis and Futuna         NaN   
          mean                                       NaN   50.530864   
          min                             American Samoa         NaN   
          sum                                        NaN         NaN   
SA        max                                  Venezuela         NaN   
          mean                                       NaN  517.041667   
          min                                  Argentina         NaN   
          sum                                        NaN         NaN   

                  population  
Continent                     
AF        max            NaN  
          mean           NaN  
          min            NaN  
          sum   3.379531e+08  
AN        max            NaN  
          mean           NaN  
          min            NaN  
          sum   1.305000e+03  
AS        max            NaN  
          mean           NaN  
          min            NaN  
          sum   1.721485e+09  
EU        max            NaN  
          mean           NaN  
          min            NaN  
          sum   7.084747e+08  
NA        max            NaN  
          mean           NaN  
          min            NaN  
          sum   4.690015e+08  
OC        max            NaN  
          mean           NaN  
          min            NaN  
          sum   4.937673e+07  
SA        max            NaN  
          mean           NaN  
          min            NaN  
          sum   2.942768e+08

In [113]:
# exemple
tab.stack(0)

max        mean  \
Continent                                                                    
AF        population                                       NaN         NaN   
          elevation                                        NaN  621.725000   
          Country                                     Zimbabwe         NaN   
AN        population                                       NaN         NaN   
          elevation                                        NaN   10.000000   
          Country     South Georgia and South Sandwich Islands         NaN   
AS        population                                       NaN         NaN   
          elevation                                        NaN  484.471519   
          Country                                        Yemen         NaN   
EU        population                                       NaN         NaN   
          elevation                                        NaN  297.007186   
          Country                                        Åland         NaN   
NA        population                                       NaN         NaN   
          elevation                                        NaN  290.176415   
          Country                                United States         NaN   
OC        population                                       NaN         NaN   
          elevation                                        NaN   50.530864   
          Country                            Wallis and Futuna         NaN   
SA        population                                       NaN         NaN   
          elevation                                        NaN  517.041667   
          Country                                    Venezuela         NaN   

                                 min           sum  
Continent                                           
AF        population             NaN  3.379531e+08  
          elevation              NaN           NaN  
          Country            Algeria           NaN  
AN        population             NaN  1.305000e+03  
          elevation              NaN           NaN  
          Country         Antarctica           NaN  
AS        population             NaN  1.721485e+09  
          elevation              NaN           NaN  
          Country        Afghanistan           NaN  
EU        population             NaN  7.084747e+08  
          elevation              NaN           NaN  
          Country            Albania           NaN  
NA        population             NaN  4.690015e+08  
          elevation              NaN           NaN  
          Country           Anguilla           NaN  
OC        population             NaN  4.937673e+07  
          elevation              NaN           NaN  
          Country     American Samoa           NaN  
SA        population             NaN  2.942768e+08  
          elevation              NaN           NaN  
          Country          Argentina           NaN

**Remarque** : la méthode `pivot_table()` peut être simulée avec un `groupby()` suivi d'un `unstack()`.

In [114]:
# remarque pivot_table = groupby + unstack
df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,1.167213e+10,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,1.023296e+09,NaN,NaN,NaN
Algeria,1.123276e+10,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,1275695.0,NaN
Andorra,NaN,NaN,NaN,9.240720e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,1.092032e+06,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,8.948588e+09,NaN,NaN,NaN,NaN
Zambia,1.162572e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# remarque pivot_table = groupby + unstack
df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

Continent,AF,AN,AS,EU,NA,OC,SA
Country,,,,,,,
Afghanistan,NaN,NaN,1.167213e+10,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,1.023296e+09,NaN,NaN,NaN
Algeria,1.123276e+10,NaN,NaN,NaN,NaN,NaN,NaN
American Samoa,NaN,NaN,NaN,NaN,NaN,1275695.0,NaN
Andorra,NaN,NaN,NaN,9.240720e+05,NaN,NaN,NaN
...,...,...,...,...,...,...,...
Western Sahara,1.092032e+06,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,8.948588e+09,NaN,NaN,NaN,NaN
Zambia,1.162572e+09,NaN,NaN,NaN,NaN,NaN,NaN


Le `pivot_table()` est même un peu plus lent, mais plus facile à comprendre.

In [117]:
# remarque pivot_table = groupby + unstack
%timeit df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

58.5 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [118]:
# groupby + unstack
%timeit df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

42.1 ms ± 685 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Dataset n° 3**

United Nations (UNCTAD) with FDI inflows (Foreign direct investment), by region and economy from
1990 to 2018.

Ce fichier est en format large avec une colonne par année, facilement lisible par un humain.

In [119]:
# UNCTAB dataset
df_un = pd.read_excel('WIR19_tab01.xlsx',
                      header=2,
                      nrows=240)
df_un = df_un.drop(0)
df_un['Region/economy'] = df_un['Region/economy'].apply(str.strip)
df_un.columns = ['Region/economy'] + [int(col) for col in df_un.columns[1:]]
df_un

,Region/economy,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,World,204886.351424,153957.259850,162916.898975,220084.505159,254910.282247,341522.543094,388825.078858,480774.095268,690861.021060,...,1.172234e+06,1.365107e+06,1.561354e+06,1.470334e+06,1.431164e+06,1.357239e+06,2.033802e+06,1.918679e+06,1.497371e+06,1.297153e+06
2,Developed economies,170166.882709,114508.768046,107809.179346,141370.315360,150589.703875,219760.864472,236347.865354,285575.605654,508697.242961,...,6.487782e+05,6.790101e+05,8.168368e+05,7.417653e+05,6.948481e+05,6.230776e+05,1.268594e+06,1.197735e+06,7.592561e+05,5.568920e+05
3,Europe,102630.292871,81748.341024,74236.381665,79026.703380,88547.808451,135587.307684,130865.901072,155722.798345,298843.192160,...,4.334056e+05,4.098058e+05,4.775278e+05,4.257446e+05,3.511968e+05,2.833924e+05,7.150168e+05,6.116934e+05,3.840234e+05,1.718776e+05
4,European Union,95560.518749,78785.458885,74506.203225,78117.977606,82405.899729,130964.690115,124492.976251,144955.924290,285814.923823,...,3.867505e+05,3.626409e+05,4.347554e+05,3.764621e+05,3.450345e+05,2.656186e+05,6.358399e+05,5.561182e+05,3.405704e+05,2.776402e+05
5,Austria,653.000000,360.000000,1485.518380,1125.106168,2075.823025,1825.089897,4323.515257,2668.974480,4571.855469,...,9.268300e+03,2.575464e+03,1.061565e+04,3.988878e+03,5.719908e+03,4.577236e+03,1.487764e+03,-8.170196e+03,1.109211e+04,7.618219e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Tajikistan,0.000000,0.000000,9.000000,9.000000,12.000000,10.000000,18.000000,18.000000,29.940400,...,1.313200e+02,1.552784e+02,2.091933e+02,2.547928e+02,2.146462e+02,4.317106e+02,5.586260e+02,3.436861e+02,2.698773e+02,3.167354e+02
236,Turkmenistan,0.000000,0.000000,0.000010,79.000000,103.000000,233.000000,108.060000,107.860000,62.300000,...,4.553000e+03,3.632300e+03,3.391067e+03,3.129615e+03,2.861421e+03,3.830131e+03,3.042968e+03,2.243160e+03,2.085944e+03,1.985147e+03
237,Ukraine,0.000000,0.000000,200.000000,200.000000,159.000000,267.000000,521.000000,623.000000,743.000000,...,4.816000e+03,6.495000e+03,7.207000e+03,8.401000e+03,4.499000e+03,4.100000e+02,2.961000e+03,3.284000e+03,2.601000e+03,2.355000e+03
238,Uzbekistan,0.000000,0.000000,9.000000,48.000000,73.000000,-24.000000,90.000000,166.800000,139.600000,...,8.420000e+02,1.636449e+03,1.635145e+03,5.630405e+02,6.347013e+02,7.573580e+02,6.648900e+01,1.341390e+02,9.765300e+01,4.124090e+02


On peut utiliser `stack()` + `reset_index()` pour le passer en format long, dit normalisé.

In [122]:
# pour le passer au format long
# on peut utiliser stack + reset_index
df_un.set_index('Region/economy').stack().reset_index()

,Region/economy,level_1,0
0,World,1990,204886.351424
1,World,1991,153957.259850
2,World,1992,162916.898975
3,World,1993,220084.505159
4,World,1994,254910.282247
...,...,...,...
6926,Georgia,2014,1817.981000
6927,Georgia,2015,1652.507000
6928,Georgia,2016,1565.838000
6929,Georgia,2017,1894.491000


In [123]:
# vérification de la longueur
len(df_un) * (df_un.shape[1] - 1)

6931

La méthode `melt()` fait aussi ça très bien.

Elle prend comme arguments :
- la liste des colonnes associées à l'identité des enregistrements
- éventuellement la liste des colonnes associées aux différentes valeurs considérées

Elle génère un DataFrame normalisé avec les colonnes associées à l'identité des enregistrements, ainsi qu'une colonne "variable" correspondant aux noms des anciennes colonnes de valeurs et une colonne "value" correspondant aux valeurs des anciennes colonnes de valeurs.

In [124]:
df_un.melt?

In [125]:
# ou bien tout simplement
# la méthode melt
tab = df_un.melt(id_vars=['Region/economy'])
tab

,Region/economy,variable,value
0,World,1990,204886.351424
1,Developed economies,1990,170166.882709
2,Europe,1990,102630.292871
3,European Union,1990,95560.518749
4,Austria,1990,653.000000
...,...,...,...
6926,Tajikistan,2018,316.735371
6927,Turkmenistan,2018,1985.147000
6928,Ukraine,2018,2355.000000
6929,Uzbekistan,2018,412.409000


In [136]:
# pour revenir pratiquement au DataFrame de départ
tab.set_index(['Region/economy', 'variable']).unstack().reset_index().tail(20)

Region/economy          value                 \
variable                                              1990           1991   
219                            Turkmenistan       0.000000       0.000000   
220                                  Tuvalu       0.000000       0.000000   
221                                  Uganda      -5.910000       1.000000   
222                                 Ukraine       0.000000       0.000000   
223                    United Arab Emirates    -115.820000      25.940000   
224                          United Kingdom   30461.116000   14846.168000   
225             United Republic of Tanzania       0.010000       0.010000   
226                           United States   48422.000000   22799.000000   
227                                 Uruguay      41.540000      32.340000   
228                              Uzbekistan       0.000000       0.000000   
229                                 Vanuatu      13.111200      25.474500   
230       Venezuela, Bolivarian Republic of     778.192000     231.159000   
231                                Viet Nam     180.000000     375.190278   
232                             West Africa    1553.427940    1367.169953   
233                               West Asia     796.413000    2132.947000   
234                                   World  204886.351424  153957.259850   
235                                   Yemen    -130.900000     282.500000   
236                     Yugoslavia (former)      67.000000     119.000000   
237                                  Zambia     202.780000      34.340000   
238                                Zimbabwe      12.200000       2.800000   

                                                                      \
variable           1992           1993           1994           1995   
219            0.000010      79.000000     103.000000     233.000000   
220            0.000000       0.000000       0.000000       0.000000   
221            3.000000      54.565000      88.152000     124.509000   
222          200.000000     200.000000     159.000000     267.000000   
223          129.720000     401.320000      62.460000     399.850000   
224        15472.801497   14804.493991    9252.755090   19969.447723   
225           12.000000      20.000000      50.000000     150.000000   
226        19222.000000   50663.000000   45095.000000   58772.000000   
227           11.300000     101.500000     154.500000     156.600000   
228            9.000000      48.000000      73.000000     -24.000000   
229           26.452900      25.967900      29.786200      31.044900   
230         1937.428366     417.553846     813.000000     985.000000   
231          473.945856     926.303715    1944.515936    1780.400000   
232         1401.331984    2121.561096    2787.390882    1860.777488   
233         3105.784000    2107.278098    1883.086685    2463.205335   
234       162916.898975  220084.505159  254910.282247  341522.543094   
235          718.000000     903.000000      15.800000    -217.700000   
236            0.000000       0.000000       0.000000       0.000000   
237           45.000000     314.400000      40.000000     107.000000   
238           19.000000      38.000000      41.000000     117.700000   

                                                       ...                \
variable           1996           1997           1998  ...          2009   
219          108.060000     107.860000      62.300000  ...  4.553000e+03   
220            0.000000       0.000000       0.000000  ...  1.751000e+00   
221          121.507000     141.500765     132.633270  ...  8.415710e+02   
222          521.000000     623.000000     743.000000  ...  4.816000e+03   
223          300.520000     232.430000     257.660000  ...  1.134288e+03   
224        24435.298413   33226.594372   74321.325419  ...  8.970914e+04   
225          148.510000     157.800000     172.200000  ...  9.526100e+02   
226        84455.000000  103398.000000  174434.000000  ...  1.436

**Exercice n° 3**

Faites la jointure de `df_un` avec `df_pays` en utilisant la fonction `merge()` ou la méthode `join()`.

Comparez le nombre de lignes entre le résultat et `df_pays`. D'où vient le problème ?

Faire le mapping avec un dictionnaire de transcodification des noms des pays (voir le fichier `mapping.py`).

Repassez en format long avec la méthode `melt()`.

In [131]:
pd.merge(df_un, df_pays, left_on="Region/economy", right_on="Country", how="inner")

,Region/economy,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2018,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Country,Capital,Area in km²,Population,Continent
0,Austria,653.00,360.0,1485.51838,1125.106168,2075.823025,1825.089897,4323.515257,2668.974480,4571.855469,...,7618.219054,AT,AUT,40,AU,Austria,Vienna,83858.0,8847037,EU
1,Belgium,0.00,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4872.578760,BE,BEL,56,BE,Belgium,Brussels,30510.0,11422068,EU
2,Bulgaria,4.00,55.9,41.50000,40.000000,105.400000,90.400000,109.000000,646.661361,678.392860,...,2058.800269,BG,BGR,100,BU,Bulgaria,Sofia,110910.0,7000039,EU
3,Croatia,0.00,0.0,13.00000,118.265145,110.332738,103.475363,485.181633,579.574957,943.651722,...,1159.415243,HR,HRV,191,HR,Croatia,Zagreb,56542.0,4089400,EU
4,Cyprus,126.62,81.8,107.40000,83.400000,75.200000,375.757040,428.898310,548.362774,345.345048,...,3284.998643,CY,CYP,196,CY,Cyprus,Nicosia,9250.0,1189265,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Tajikistan,0.00,0.0,9.00000,9.000000,12.000000,10.000000,18.000000,18.000000,29.940400,...,316.735371,TJ,TJK,762,TI,Tajikistan,Dushanbe,143100.0,9100837,AS
177,Turkmenistan,0.00,0.0,0.00001,79.000000,103.000000,233.000000,108.060000,107.860000,62.300000,...,1985.147000,TM,TKM,795,TX,Turkmenistan,Ashgabat,488100.0,5850908,AS
178,Ukraine,0.00,0.0,200.00000,200.000000,159.000000,267.000000,521.000000,623.000000,743.000000,...,2355.000000,UA,UKR,804,UP,Ukraine,Kyiv,603700.0,44622516,EU
179,Uzbekistan,0.00,0.0,9.00000,48.000000,73.000000,-24.000000,90.000000,166.800000,139.600000,...,412.409000,UZ,UZB,860,UZ,Uzbekistan,Tashkent,447400.0,32955400,AS


In [132]:
len(df_pays)

250

In [134]:
set(df_un["Region/economy"]).symmetric_difference(set(df_pays["Country"]))

{'Africa',
 'American Samoa',
 'Andorra',
 'Antarctica',
 'Asia',
 'Belgium and Luxembourg',
 'Bolivia',
 'Bolivia, Plurinational State of',
 'Bonaire, Sint Eustatius, and Saba',
 'Bouvet Island',
 'British Indian Ocean Territory',
 'Brunei',
 'Brunei Darussalam',
 'CIS',
 'Caribbean',
 'Central Africa',
 'Central America',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Congo',
 'Congo Republic',
 'Congo, Democratic Republic of the',
 'Cuba',
 "Côte d'Ivoire",
 'DR Congo',
 'Developed economies',
 'Developing economies',
 'East Africa',
 'East Asia',
 'East and South-East Asia',
 'Europe',
 'European Union',
 'Falkland Islands',
 'Faroe Islands',
 'French Southern Territories',
 'Gambia',
 'Gibraltar',
 'Greenland',
 'Guadeloupe',
 'Guam',
 'Guernsey',
 'Heard and McDonald Islands',
 'Hong Kong',
 'Hong Kong, China',
 'Iran',
 'Iran, Islamic Republic of',
 'Isle of Man',
 'Ivory Coast',
 'Jersey',
 "Korea, Democratic People's Republic of",
 'Korea, Republic of',
 'Kosovo',
 "Lao Pe

In [ ]:
set.

In [135]:
from country_mapping import mapping
mapping

{'Bolivia, Plurinational State of': 'Bolivia',
 'Brunei Darussalam': 'Brunei',
 'Czech Republic': 'Czechia',
 'Congo': 'Congo Republic',
 'Congo, Democratic Republic of the': 'DR Congo',
 "Côte d'Ivoire": 'Ivory Coast',
 'Gambia': 'The Gambia',
 'Hong Kong, China': 'Hong Kong',
 'Iran, Islamic Republic of': 'Iran',
 "Korea, Democratic People's Republic of": 'North Korea',
 'Korea, Republic of': 'South Korea',
 "Lao People's Democratic Republic": 'Laos',
 'Macao, China': 'Macao',
 'Micronesia, Federated States of': 'Micronesia',
 'Moldova, Republic of': 'Moldova',
 'Reunion': 'Réunion',
 'Russian Federation': 'Russia',
 'Saint Kitts and Nevis': 'St Kitts and Nevis',
 'Saint Vincent and the Grenadines': 'St Vincent and Grenadines',
 'Sao Tome and Principe': 'São Tomé and Príncipe',
 'State of Palestine': 'Palestine',
 'Syrian Arab Republic': 'Syria',
 'Taiwan Province of China': 'Taiwan',
 'United Republic of Tanzania': 'Tanzania',
 'Venezuela, Bolivarian Republic of': 'Venezuela',
 'Vie

In [148]:
len(mapping)

26

In [144]:
mapping.get('France', "France")

'France'

In [145]:
df_un["Country"] = df_un["Region/economy"].apply(lambda x: mapping.get(x, x))

In [147]:
pd.merge(df_un, df_pays, on="Country")

,Region/economy,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2018,Country,ISO-3166alpha2,ISO-3166alpha3,ISO-3166numeric,fips,Capital,Area in km²,Population,Continent
0,Austria,653.00,360.0,1485.51838,1125.106168,2075.823025,1825.089897,4323.515257,2668.974480,4571.855469,...,7618.219054,Austria,AT,AUT,40,AU,Vienna,83858.0,8847037,EU
1,Belgium,0.00,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4872.578760,Belgium,BE,BEL,56,BE,Brussels,30510.0,11422068,EU
2,Bulgaria,4.00,55.9,41.50000,40.000000,105.400000,90.400000,109.000000,646.661361,678.392860,...,2058.800269,Bulgaria,BG,BGR,100,BU,Sofia,110910.0,7000039,EU
3,Croatia,0.00,0.0,13.00000,118.265145,110.332738,103.475363,485.181633,579.574957,943.651722,...,1159.415243,Croatia,HR,HRV,191,HR,Zagreb,56542.0,4089400,EU
4,Cyprus,126.62,81.8,107.40000,83.400000,75.200000,375.757040,428.898310,548.362774,345.345048,...,3284.998643,Cyprus,CY,CYP,196,CY,Nicosia,9250.0,1189265,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Tajikistan,0.00,0.0,9.00000,9.000000,12.000000,10.000000,18.000000,18.000000,29.940400,...,316.735371,Tajikistan,TJ,TJK,762,TI,Dushanbe,143100.0,9100837,AS
202,Turkmenistan,0.00,0.0,0.00001,79.000000,103.000000,233.000000,108.060000,107.860000,62.300000,...,1985.147000,Turkmenistan,TM,TKM,795,TX,Ashgabat,488100.0,5850908,AS
203,Ukraine,0.00,0.0,200.00000,200.000000,159.000000,267.000000,521.000000,623.000000,743.000000,...,2355.000000,Ukraine,UA,UKR,804,UP,Kyiv,603700.0,44622516,EU
204,Uzbekistan,0.00,0.0,9.00000,48.000000,73.000000,-24.000000,90.000000,166.800000,139.600000,...,412.409000,Uzbekistan,UZ,UZB,860,UZ,Tashkent,447400.0,32955400,AS
